In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore") #알림 무시

In [29]:
movies = pd.read_csv('/content/movies.csv')

In [40]:
ratings = pd.read_csv('/content/ratings.csv')
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'],unit = 's')

ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04
3,1,47,5.0,2000-07-30 19:03:35
4,1,50,5.0,2000-07-30 18:48:51


In [59]:
user_movie_ratings = ratings.pivot_table(index='userId', columns='movieId', values='rating')

In [73]:
user_movie_ratings.fillna(0, inplace=True)

user_movie_ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
X = user_movie_ratings.copy()
y = user_movie_ratings.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y)

model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [90]:
recommendation_results = []

In [91]:
for user_id in X_test.index.unique():
    user_ratings = user_movie_ratings.loc[user_id]
    user_unrated_movies = user_ratings[user_ratings == 0].index

    if len(user_unrated_movies) == 0:
        continue

    predicted_ratings = model.predict([user_movie_ratings.loc[user_id]])[0]

    movie_indices = [user_movie_ratings.columns.get_loc(movie_id) for movie_id in user_unrated_movies]

    if len(movie_indices) >= 3:
        top_n_movie_indices = predicted_ratings[movie_indices].argsort()[-3:][::-1]

        recommended_movie_ids = [user_unrated_movies[i] for i in top_n_movie_indices]
        recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]

        if not recommended_movies.empty:
            movie_titles = recommended_movies['title'].values.tolist()
            recommendation_results.append({
                'userId': user_id,
                '추천영화1': movie_titles[0] if len(movie_titles) > 0 else '',
                '추천영화2': movie_titles[1] if len(movie_titles) > 1 else '',
                '추천영화3': movie_titles[2] if len(movie_titles) > 2 else ''
            })

In [92]:
recommendation_df = pd.DataFrame(recommendation_results, columns=['userId', '추천영화1', '추천영화2', '추천영화3'])

In [93]:
recommendation_df.head()

,userId,추천영화1,추천영화2,추천영화3
0,11,"English Patient, The (1996)","Hunt for Red October, The (1990)",L.A. Confidential (1997)
1,134,Johnny Mnemonic (1995),Legends of the Fall (1994),Outbreak (1995)
2,541,Clerks (1994),What's Eating Gilbert Grape (1993),While You Were Sleeping (1995)
3,69,Trainspotting (1996),Contact (1997),Total Recall (1990)
4,366,"Lord of the Rings: The Two Towers, The (2002)","Prestige, The (2006)",Up (2009)
